# Managing Ansys-CFX cases

In [ ]:
from cfdtoolkit.cfx.case import CFXCase
from cfdtoolkit import set_loglevel
set_loglevel('INFO')

## Steady-state CFX Case

A `CFXCase` is initialized with the `.cfx` file. The solver file (`.def`-file) is automatically detected as well as the `.res`-files if available.

In [ ]:
from cfdtoolkit.cfx.session import CFX5PRE

In [ ]:
case_steady = CFXCase('../../testdata/cylinderflow/steady_state/cyl_steadystate_laminar.cfx')
case_steady

In [ ]:
case_steady.working_dir

In [ ]:
if len(case_steady.res_files) > 0:
    case_steady.reset()

### Starting a case
A case can be started from any other initial result file or from none. The number of processors to be used and the clock time can be passed as arguments. `start()` will return the command line that was constructed and executed.

Note, that when passing `wait=True` python calls the CFX solver and waits until it finishes. Only after that the next python command can be executed.

In [ ]:
case_steady = CFXCase('../../testdata/cylinderflow/steady_state/cyl_steadystate_laminar.cfx')
case_steady

In [ ]:
case_steady.ccl.flow[0].max_iterations = 5

In [ ]:
case_steady.ccl.dump()

update the path to the inlet velocity data:

In [ ]:
import pathlib
inlet_data_source = pathlib.Path("../../testdata/cylinderflow/inlet_profile_Re20.csv").resolve()
case_steady.ccl['LIBRARY/CEL/FUNCTION: inlet/DATA SOURCE'].options['File Name'] = str(inlet_data_source)
case_steady.ccl.dump()
case_steady.ccl.filename

In [ ]:
case_steady.ccl.flow[0].dump()

In [ ]:
del case_steady.ccl.expressions.options['dmfr']
del case_steady.ccl.flow[0].monitor_object['dmfr']

In [ ]:
if True:
    case_steady.start(initial_result_file=None, nproc=2, timeout=120, wait=True)

### Resume a case
You can call `.resume()` on your latest result to continue from it. Note, calling `resume` on the latest result file will not take changes in the ccl file into account! Calling it from the `CFXCase` instance however will! You may call `.update_from_ccl()` to be absolutely sure!

In [ ]:
case_steady.update_from_ccl()
case_steady.latest.resume(nproc=2)

# # or call:
# case_steady.resume(nproc=2)

### Stopping a case
You can call `.stop()` from your `CFXCase` instance to stop the run(s). This will btw. stop all running cases in the working directory!

In [ ]:
case_steady.stop()  # touches a file "stp" in the *.dir directory if exists

## Monitor Points and Expressions

User points are very helpful to analyze the runs

### Define an expression:

In [ ]:
case_steady.ccl.expressions.options['test'] = 'test'

In [ ]:
del case_steady.ccl.expressions.options['test']
del case_steady.ccl.expressions.options['dmfr']  # delete if exists from previous run for the sake of this tutorial

In [ ]:
case_steady.ccl.expressions.options['dmfr'] = 'massFlow()@OUTLET-massFlow()@INLET'

In [ ]:
case_steady.ccl.expressions.options

### Defining a monitor point:

In [ ]:
from cfdtoolkit.cfx.core import MonitorObject
case_steady.ccl.flow[0].monitor_object['dmfr'] = MonitorObject(expression_value='dmfr', coord_frame='Coord 0')
case_steady.ccl.dump()

Let's continue/resume the run as we have now a new expression set and registered it as a monitor point

In [ ]:
case_steady.resume(nproc=2)

### User Points

The example case has some user points registered which can be requested by calling the `Monitor` instance of e.g. the lates result file:

In [ ]:
case_steady.latest

In [ ]:
case_steady.latest.monitor.user_points.keys()

In [ ]:
case_steady.latest.monitor.user_points['dmfr'].plot()

In [ ]:
#if case_steady.latest:
print(len(case_steady.result_files.cfx_res_files))
print(case_steady.latest)

## Controll and change case settings

The following shows how to set and read various options in your case file.

In [ ]:
case_steady.ccl

In [ ]:
case_steady.ccl.flow[0].domains[0].get_boundary_type('inlet').dump()

In [ ]:
case_steady.ccl.to_ccl('test.ccl')

In [ ]:
#case_steady.import_ccl()

In [ ]:
case_steady.ccl.flow[0].domains[0].get_boundary_type('inlet').dump()

### Setting a new boundary condition

In [ ]:
from cfdtoolkit.cfx.boundary_conditions.inlet import NormalSpeed

In [ ]:
inlet = case_steady.ccl.flow[0].domains[0].get_boundary_type('inlet')

In [ ]:
inlet.condition = NormalSpeed(flow_regime='Subsonic', normal_speed=1)

In [ ]:
inlet.condition

In [ ]:
case_steady.ccl.flow[0].domains[0].get_boundary_type('inlet').dump()

In [ ]:
case_steady.update()